# **Chapter 2: Next-Generation Sequencing**
</br>

**Performing basic sequence analysis**

**Goal:** working with *FASTA* files and do some manipulation, such as reverse complementing or transcription. 
</br>
We will use the **human Lactase (LCT) gene** as an example.

firstly, we need to import *Entrez, SeqIO* from **BioPython** package.
</br>
`Entrez` : for search and fetch data from NCBI databases
</br>
`SeqIO` : Sequence input/output
</br>
**Note:** Do not forget to put in the correct email address.

In [4]:
from Bio import Entrez, SeqIO
Entrez.email = 'a.doustmohammadi@modares.ac.ir'

**how we can search and fetch data from NCBI databases?**

**First Step:** search on the nucleotide database
</br>
to do that, we should use: `Entrez.esearch()` function
</br>
`Entrez.esearch()` arguments: 
1.   db: database name
2.   term: ncbi search query (for more detail, see: https://www.ncbi.nlm.nih.gov/nuccore/advanced)
3. rettype: Retrieval type


In [77]:
hdl = Entrez.efetch(db='nucleotide', id=['NM_002299'], rettype='fasta') #Lactase gene

**Next Step:** read the result that is returned
</br>
to do that, we should use: `SeqIO.read()` function
</br>
outputs of `Entrez.read()` function:

In [78]:
seq = SeqIO.read(hdl, 'fasta')
print(seq)
# only sequence
print(seq.seq)

ID: NM_002299.4
Name: NM_002299.4
Description: NM_002299.4 Homo sapiens lactase (LCT), mRNA
Number of features: 0
Seq('AACAGTTCCTAGAAAATGGAGCTGTCTTGGCATGTAGTCTTTATTGCCCTGCTA...GTC')
AACAGTTCCTAGAAAATGGAGCTGTCTTGGCATGTAGTCTTTATTGCCCTGCTAAGTTTTTCATGCTGGGGGTCAGACTGGGAGTCTGATAGAAATTTCATTTCCACCGCTGGTCCTCTAACCAATGACTTGCTGCACAACCTGAGTGGTCTCCTGGGAGACCAGAGTTCTAACTTTGTAGCAGGGGACAAAGACATGTATGTTTGTCACCAGCCACTGCCCACTTTCCTGCCAGAATACTTCAGCAGTCTCCATGCCAGTCAGATCACCCATTATAAGGTATTTCTGTCATGGGCACAGCTCCTCCCAGCAGGAAGCACCCAGAATCCAGACGAGAAAACAGTGCAGTGCTACCGGCGACTCCTCAAGGCCCTCAAGACTGCACGGCTTCAGCCCATGGTCATCCTGCACCACCAGACCCTCCCTGCCAGCACCCTCCGGAGAACCGAAGCCTTTGCTGACCTCTTCGCCGACTATGCCACATTCGCCTTCCACTCCTTCGGGGACCTAGTTGGGATCTGGTTCACCTTCAGTGACTTGGAGGAAGTGATCAAGGAGCTTCCCCACCAGGAATCAAGAGCGTCACAACTCCAGACCCTCAGTGATGCCCACAGAAAAGCCTATGAGATTTACCACGAAAGCTATGCTTTTCAGGGCGGAAAACTCTCTGTTGTCCTGCGAGCTGAAGATATCCCGGAGCTCCTGCTAGAACCACCCATATCTGCGCTTGCCCAGGACACGGTCGATTTCCTCTCTCTTGATTTGTCTTATGAATGCCAAAATGAGGCAAGTCTGCGGCAGAAGCTGAGTAAATTGCA

let's start by **saving it to a FASTA file on our local disk**:

In [79]:
w_hdl = open('Lactase gene.fasta', 'w')
w_seq = seq
SeqIO.write([w_seq], w_hdl, 'fasta')
w_hdl.close()

If you want to save part of the sequence (e.g: nucleotide 11 to nucleotide 5795):

In [71]:
w_hdl = open('Lactase gene_11_5795.fasta', 'w')
w_seq = seq[11:5795]
SeqIO.write([w_seq], w_hdl, 'fasta')
w_hdl.close()

The `SeqIO.write()` function takes a **list of sequences to write** (not just a single
one). Be careful with this idiom. 
</br>
If you want to write many sequences (and you
could easily write millions with NGS), do not use a list (as shown in the preceding
code), because **this will allocate massive amounts of memory**.
</br>
 Either use an
iterator, or use the SeqIO.write function several times with a subset of the
sequence on each write.

So, It's better to use the following style:

In [72]:
w_hdl = open('Lactase gene final Style.fasta', 'w')
sequence_length = len(seq)
length_of_each_part = 1000    
number_of_part = int(sequence_length/1000)
for i in range(number_of_part):
  first_nucleotide_index = i*length_of_each_part
  last_nucleotide_index = (i+1)*length_of_each_part
  w_seq = seq[first_nucleotide_index:last_nucleotide_index ]
  SeqIO.write([w_seq], w_hdl, 'fasta')
# If we haven't reached the end of the sequence yet:
if sequence_length > last_nucleotide_index:  
  w_seq = seq[last_nucleotide_index:sequence_length]
  SeqIO.write([w_seq], w_hdl, 'fasta')  
w_hdl.close()

**How to read FASTA file from local disk?**

In [80]:
recs = SeqIO.parse('Lactase gene.fasta', 'fasta')
for rec in recs:
 seq = rec.seq
 print(seq)
 print(rec.description)
 print(seq[:10])  # The sequence object can use typical array slices to get part of a sequence.

AACAGTTCCTAGAAAATGGAGCTGTCTTGGCATGTAGTCTTTATTGCCCTGCTAAGTTTTTCATGCTGGGGGTCAGACTGGGAGTCTGATAGAAATTTCATTTCCACCGCTGGTCCTCTAACCAATGACTTGCTGCACAACCTGAGTGGTCTCCTGGGAGACCAGAGTTCTAACTTTGTAGCAGGGGACAAAGACATGTATGTTTGTCACCAGCCACTGCCCACTTTCCTGCCAGAATACTTCAGCAGTCTCCATGCCAGTCAGATCACCCATTATAAGGTATTTCTGTCATGGGCACAGCTCCTCCCAGCAGGAAGCACCCAGAATCCAGACGAGAAAACAGTGCAGTGCTACCGGCGACTCCTCAAGGCCCTCAAGACTGCACGGCTTCAGCCCATGGTCATCCTGCACCACCAGACCCTCCCTGCCAGCACCCTCCGGAGAACCGAAGCCTTTGCTGACCTCTTCGCCGACTATGCCACATTCGCCTTCCACTCCTTCGGGGACCTAGTTGGGATCTGGTTCACCTTCAGTGACTTGGAGGAAGTGATCAAGGAGCTTCCCCACCAGGAATCAAGAGCGTCACAACTCCAGACCCTCAGTGATGCCCACAGAAAAGCCTATGAGATTTACCACGAAAGCTATGCTTTTCAGGGCGGAAAACTCTCTGTTGTCCTGCGAGCTGAAGATATCCCGGAGCTCCTGCTAGAACCACCCATATCTGCGCTTGCCCAGGACACGGTCGATTTCCTCTCTCTTGATTTGTCTTATGAATGCCAAAATGAGGCAAGTCTGCGGCAGAAGCTGAGTAAATTGCAGACCATTGAGCCAAAAGTGAAAGTTTTCATCTTCAACCTAAAACTCCCAGACTGCCCCTCCACCATGAAGAACCCAGCCAGTCTGCTCTTCAGCCTTTTTGAAGCCATAAATAAAGACCAAGTGCTCACCATTGGGTTTGATATTAATGAGTTTCTGAGTTGTTCATCAAGTTCCAAGAAAA

**Now, we can transcribe DNA as follows:**

In [81]:
rna = seq.transcribe()
print(rna)

AACAGUUCCUAGAAAAUGGAGCUGUCUUGGCAUGUAGUCUUUAUUGCCCUGCUAAGUUUUUCAUGCUGGGGGUCAGACUGGGAGUCUGAUAGAAAUUUCAUUUCCACCGCUGGUCCUCUAACCAAUGACUUGCUGCACAACCUGAGUGGUCUCCUGGGAGACCAGAGUUCUAACUUUGUAGCAGGGGACAAAGACAUGUAUGUUUGUCACCAGCCACUGCCCACUUUCCUGCCAGAAUACUUCAGCAGUCUCCAUGCCAGUCAGAUCACCCAUUAUAAGGUAUUUCUGUCAUGGGCACAGCUCCUCCCAGCAGGAAGCACCCAGAAUCCAGACGAGAAAACAGUGCAGUGCUACCGGCGACUCCUCAAGGCCCUCAAGACUGCACGGCUUCAGCCCAUGGUCAUCCUGCACCACCAGACCCUCCCUGCCAGCACCCUCCGGAGAACCGAAGCCUUUGCUGACCUCUUCGCCGACUAUGCCACAUUCGCCUUCCACUCCUUCGGGGACCUAGUUGGGAUCUGGUUCACCUUCAGUGACUUGGAGGAAGUGAUCAAGGAGCUUCCCCACCAGGAAUCAAGAGCGUCACAACUCCAGACCCUCAGUGAUGCCCACAGAAAAGCCUAUGAGAUUUACCACGAAAGCUAUGCUUUUCAGGGCGGAAAACUCUCUGUUGUCCUGCGAGCUGAAGAUAUCCCGGAGCUCCUGCUAGAACCACCCAUAUCUGCGCUUGCCCAGGACACGGUCGAUUUCCUCUCUCUUGAUUUGUCUUAUGAAUGCCAAAAUGAGGCAAGUCUGCGGCAGAAGCUGAGUAAAUUGCAGACCAUUGAGCCAAAAGUGAAAGUUUUCAUCUUCAACCUAAAACUCCCAGACUGCCCCUCCACCAUGAAGAACCCAGCCAGUCUGCUCUUCAGCCUUUUUGAAGCCAUAAAUAAAGACCAAGUGCUCACCAUUGGGUUUGAUAUUAAUGAGUUUCUGAGUUGUUCAUCAAGUUCCAAGAAAA

**Finally, we can translate our gene into a protein:**

In [82]:
prot = seq.translate()
print(prot)

NSS*KMELSWHVVFIALLSFSCWGSDWESDRNFISTAGPLTNDLLHNLSGLLGDQSSNFVAGDKDMYVCHQPLPTFLPEYFSSLHASQITHYKVFLSWAQLLPAGSTQNPDEKTVQCYRRLLKALKTARLQPMVILHHQTLPASTLRRTEAFADLFADYATFAFHSFGDLVGIWFTFSDLEEVIKELPHQESRASQLQTLSDAHRKAYEIYHESYAFQGGKLSVVLRAEDIPELLLEPPISALAQDTVDFLSLDLSYECQNEASLRQKLSKLQTIEPKVKVFIFNLKLPDCPSTMKNPASLLFSLFEAINKDQVLTIGFDINEFLSCSSSSKKSMSCSLTGSLALQPDQQQDHETTDSSPASAYQRIWEAFANQSRAERDAFLQDTFPEGFLWGASTGAFNVEGGWAEGGRGVSIWDPRRPLNTTEGQATLEVASDSYHKVASDVALLCGLRAQVYKFSISWSRIFPMGHGSSPSLPGVAYYNKLIDRLQDAGIEPMATLFHWDLPQALQDHGGWQNESVVDAFLDYAAFCFSTFGDRVKLWVTFHEPWVMSYAGYGTGQHPPGISDPGVASFKVAHLVLKAHARTWHHYNSHHRPQQQGHVGIVLNSDWAEPLSPERPEDLRASERFLHFMLGWFAHPVFVDGDYPATLRTQIQQMNRQCSHPVAQLPEFTEAEKQLLKGSADFLGLSHYTSRLISNAPQNTCIPSYDTIGGFSQHVNHVWPQTSSSWIRVVPWGIRRLLQFVSLEYTRGKVPIYLAGNGMPIGESENLFDDSLRVDYFNQYINEVLKAIKEDSVDVRSYIARSLIDGFEGPSGYSQRFGLHHVNFSDSSKSRTPRKSAYFFTSIIEKNGFLTKGAKRLLPPNTVNLPSKVRAFTFPSEVPSKAKVVWEKFSSQPKFERDLFYHGTFRDDFLWGVSSSAYQIEGAWDADGKGPSIWDNFTHTPGSNVKDNATGDIACDSYHQLDADLNMLRALKVKAYRFSISWSRIFPTGRNSSINSH